### 引入库文件

In [26]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import gc

### 参数与随机数种子

In [27]:
config = {
    'val_ratio': 0.05,
    'batch_size': 128,
    'device': 'cuda',
    'n_epoch': 30,
    'learning_rate': 0.0001,
    'model_path': './model.ckpt',
}


def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

### 处理数据


In [28]:
print('读取数据...')

data_root = '../data/hw2/'
train_data = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test_data = np.load(data_root + 'test_11.npy')

print('训练数据大小为： {}'.format(train_data.shape))
print('测试数据大小为： {}'.format(test_data.shape))

读取数据...
训练数据大小为： (1229932, 429)
测试数据大小为： (451552, 429)


In [29]:
class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


In [30]:
VAL_RATIO = config['val_ratio']

percent = int(train_data.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train_data[:percent], train_label[:percent], \
                                 train_data[percent:], train_label[percent:]
print('训练集大小： {}'.format(train_x.shape))
print('验证集大小： {}'.format(val_x.shape))

训练集大小： (1168435, 429)
验证集大小： (61497, 429)


In [31]:
BATCH_SIZE = config['batch_size']
train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

C:\Users\Shinelon\AppData\Local\Temp/ipykernel_13904/4124245113.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y.astype(np.int)


In [32]:
# 释放不需要的变量
del train_data, train_label, train_x, train_y, val_x, val_y
gc.collect()

57

### 模型模块

In [33]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 2048)
        self.layer2 = nn.Linear(2048, 2048)
        self.layer3 = nn.Linear(2048, 1024)
        self.layer4 = nn.Linear(1024, 1024)
        self.layer5 = nn.Linear(1024, 512)
        self.layer6 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39)

        self.act_fn = nn.ReLU()
        self.drop = nn.Dropout(0.5)

        self.bn1 = nn.BatchNorm1d(2048)
        self.bn2 = nn.BatchNorm1d(2048)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.bn5 = nn.BatchNorm1d(512)
        self.bn6 = nn.BatchNorm1d(128)


    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer4(x)
        x = self.bn4(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer5(x)
        x = self.bn5(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer6(x)
        x = self.bn6(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.out(x)

        return x


### 训练

In [35]:
same_seeds(233)
device = config['device']
n_epoch = config['n_epoch']
learning_rate = config['learning_rate']
model_path = config['model_path']

model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
best_acc = 0.0
for epoch in range(n_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train()
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1)  # 输出39个概率，取最大的
        batch_loss.backward()
        optimizer.step()

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    if len(val_set) > 0:
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)
                _, val_pred = torch.max(outputs, 1)

                val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
                val_loss += batch_loss.item()
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, n_epoch, train_acc / len(train_set), train_loss / len(train_loader),
                val_acc / len(val_set), val_loss / len(val_loader)
            ))

            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc / len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, n_epoch, train_acc / len(train_set), train_loss / len(train_loader)
        ))

[001/030] Train Acc: 0.474353 Loss: 1.842455 | Val Acc: 0.620973 loss: 1.260234
saving model with acc 0.621
[002/030] Train Acc: 0.577377 Loss: 1.443641 | Val Acc: 0.664699 loss: 1.103806
saving model with acc 0.665
[003/030] Train Acc: 0.606046 Loss: 1.336517 | Val Acc: 0.684521 loss: 1.033905
saving model with acc 0.685
[004/030] Train Acc: 0.623466 Loss: 1.272803 | Val Acc: 0.694814 loss: 0.989407
saving model with acc 0.695
[005/030] Train Acc: 0.635239 Loss: 1.228336 | Val Acc: 0.702099 loss: 0.957638
saving model with acc 0.702
[006/030] Train Acc: 0.645314 Loss: 1.193012 | Val Acc: 0.706262 loss: 0.937179
saving model with acc 0.706
[007/030] Train Acc: 0.652792 Loss: 1.164505 | Val Acc: 0.712051 loss: 0.915804
saving model with acc 0.712
[008/030] Train Acc: 0.659561 Loss: 1.139586 | Val Acc: 0.716767 loss: 0.899467
saving model with acc 0.717
[009/030] Train Acc: 0.664224 Loss: 1.120646 | Val Acc: 0.718734 loss: 0.882219
saving model with acc 0.719
[010/030] Train Acc: 0.66892

### 测试模块


In [37]:
test_set = TIMITDataset(test_data, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)
            
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))